<a href="https://colab.research.google.com/github/iqra-1/civic-agent/blob/main/localgov_ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏛️ LocalGov AI Agent — Dublin City Council Assistant
> End-to-end agentic RAG system using free LLMs (Mistral-7B), scraped council data, and multi-agent reasoning.

## 1. Setup & Dependencies

In [1]:
# Step 1: Uninstall conflicting packages
!pip uninstall -y transformers tokenizers langchain langchain-core langchain-community crewai llama-cpp-python numpy scipy

# Step 2: Install compatible NumPy and SciPy FIRST
!pip install -q "numpy>=2.0,<2.3" "scipy>=1.13.0"

# Step 3: Install everything else
!pip install -q \
    "transformers>=4.44.0" \
    "tokenizers>=0.19.1" \
    "sentence-transformers>=3.0.1" \
    "faiss-cpu>=1.8.0" \
    "langchain-core>=1.0.0" \
    "langchain-text-splitters>=0.2.4" \
    "langchain>=0.2.16" \
    "langchain-community>=0.2.16" \
    "crewai>=0.121.1" \
    "llama-cpp-python>=0.2.87" \
    "gradio>=4.40.0" \
    "beautifulsoup4" \
    "requests" \
    "accelerate" \
    "protobuf"

print("✅ Installation complete! Now restart the runtime.")
print("⚠️  Go to: Runtime → Restart runtime")
print("Then re-run your imports.")

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
Found existing installation: langchain 1.2.0
Uninstalling langchain-1.2.0:
  Successfully uninstalled langchain-1.2.0
Found existing installation: langchain-core 1.2.1
Uninstalling langchain-core-1.2.1:
  Successfully uninstalled langchain-core-1.2.1
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.16.3
Uninstalling scipy-1.16.3:
  Successfully uninstalled scipy-1.16.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ! pip install --upgrade transformers accelerate protobuf


In [1]:
# Restart runtime if needed (Colab sometimes requires it after llama-cpp install)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## 2. Enable GPU & Download Mistral-7B (GGUF)

In [2]:
import subprocess
import sys

In [3]:
!nvidia-smi


Wed Dec 24 13:24:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
MODEL_URL = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf"
MODEL_PATH = "/content/mistral-7b-instruct-v0.2.Q5_K_M.gguf"

if not os.path.exists(MODEL_PATH):
    !wget -O {MODEL_PATH} {MODEL_URL}


## 3. Load Local LLM (with GPU offload)

In [5]:
from llama_cpp import Llama

llm = Llama(
    model_path=MODEL_PATH,
    n_gpu_layers=-1,  # offload all layers to GPU
    n_ctx=4096,
    n_threads=8,
    verbose=False
)

# Test
response = llm.create_chat_completion(
    messages=[{"role": "user", "content": "What is Dublin City Council?"}],
    max_tokens=100
)
print(response['choices'][0]['message']['content'])

llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


 Dublin City Council is the local authority responsible for governing and managing the city of Dublin, Ireland. It is one of the 31 local authorities in Ireland and provides a range of services to the people living in its administrative area, which includes the city center and its suburbs. The council's main functions include housing, planning and development, roads and transportation, waste management, cultural and community services, and economic development. The council is elected every five years and is headed by a Lord Mayor,


## 4. Scrape Dublin City Council Pages

In [6]:
import requests
from bs4 import BeautifulSoup
import os

def scrape_page(url, filename):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    # Remove scripts/styles
    for script in soup(["script", "style"]):
        script.decompose()
    text = soup.get_text(separator="\n", strip=True)
    with open(f"/content/data/{filename}.txt", "w") as f:
        f.write(text)
    print(f"✅ Saved: {filename}")

In [7]:
# Create data dir
os.makedirs("/content/data", exist_ok=True)

# Scrape key pages
pages = {
    "waste_recycling": "https://www.dublincity.ie/residential/waste-and-recycling",
    "housing_support": "https://www.dublincity.ie/residential/housing",
    "parking_permits": "https://www.dublincity.ie/residential/parking"
}

for name, url in pages.items():
    try:
        scrape_page(url, name)
    except Exception as e:
        print(f"❌ Failed {name}: {e}")


✅ Saved: waste_recycling
✅ Saved: housing_support
✅ Saved: parking_permits


## 5. Build FAISS RAG Index

In [9]:
!pip install unstructured

In [10]:
!pip install sentence-transformers

In [11]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Load all scraped text
loader = DirectoryLoader("/content/data/", glob="*.txt")
docs = loader.load()

# Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# Embed & store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(k=3)

# Test retriever
print("📄 Sample retrieval:")
print(retriever.invoke("bin collection exemption for students")[0].page_content[:300])


/tmp/ipython-input-3359451929.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

📄 Sample retrieval:
Terms and Conditions

Sitemap

>

>

Statutory Obligations

Freedom of Information

GDPR and Data Protection

Request Information on the Environment

Protected Disclosure Procedures

Lobbying

Our Commitment to the Irish Language

Councillor Ethics

Public Sector Duty

Bye Laws

Procurement Policy a


## 6. Simulate Multi-Agent Workflow (CrewAI-style)

In [12]:
def researcher(query):
    context = " ".join([doc.page_content for doc in retriever.invoke(query)])
    prompt = f"""You are a Dublin City Council policy researcher.
Use ONLY the following context to answer the question.

Context: {context}

Question: {query}

Answer (cite facts only):"""
    return llm(prompt, max_tokens=256, stop=["\n\n"])["choices"][0]["text"].strip()




def validator(research_output, query):
    prompt = f"""Based on this research: "{research_output}",
is the citizen eligible for the requested service? Answer YES/NO and reason briefly.

Citizen query: {query}"""
    return llm(prompt, max_tokens=100)["choices"][0]["text"].strip()


def actioner(validated, query):
    if "YES" in validated:
        prompt = f"""Generate clear next steps for the citizen.
Be specific: mention forms, emails, or documents needed.

Citizen query: {query}"""
    else:
        prompt = f"""Politely explain why the citizen is not eligible and suggest alternatives.

Citizen query: {query}"""
    return llm(prompt, max_tokens=200)["choices"][0]["text"].strip()



In [13]:
def localgov_agent(query):
    research = researcher(query)
    validation = validator(research, query)
    action = actioner(validation, query)
    return {
        "research": research,
        "validation": validation,
        "final_answer": action
    }

# Test
test_query = "I'm a student in Harold's Cross. Can I get a bin collection exemption?"
result = localgov_agent(test_query)
print("🔍 RESEARCH:\n", result["research"])
print("\n✅ VALIDATION:\n", result["validation"])
print("\n📬 FINAL ANSWER:\n", result["final_answer"])


🔍 RESEARCH:
 According to Dublin City Council's Terms and Conditions under the "Waste and Recycling" section, bin collection exemptions for students are subject to the following conditions:

✅ VALIDATION:
 Answer: YES, but the eligibility is subject to the conditions outlined in Dublin City Council's Terms and Conditions under the "Waste and Recycling" section. These conditions may include proof of student status and residency in a specific area. It is recommended that the citizen checks the exact requirements with Dublin City Council.

📬 FINAL ANSWER:
 Assistant: To apply for a bin collection exemption in Harold's Cross, you'll need to contact your local authority, Dublin City Council, for assistance. You can reach them at [01 222 2222](ntouch://call/01%20222%202222 "Call 01 222 2222 using Sorenson VRS") or via email at waste.enquiries@dublincity.ie.

In your request, please mention that you are a student and provide your student ID number or other proof of student status. You may als

## 7. Launch Gradio Demo

In [4]:
import gradio as gr

def chat_interface(user_query):
    res = localgov_agent(user_query)
    return res["final_answer"]

demo = gr.Interface(
    fn=chat_interface,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Ask anything about Dublin City Council services...",
        label="Your Question"
    ),
    outputs=gr.Textbox(
        lines=8,          # taller output
        max_lines=20,     # allows scrolling if very long
        label="Council AI Response"
    ),
    title="🏛️ LocalGov AI Agent",
    description="An open-source agentic assistant for Dublin residents."
)

# Launch (public link will appear)
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://77affcada9decd9731.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
